In [ ]:
# ollama serve
# ollama pull nomic-embed-text
# ollama run jyan1/paligemma-mix-224
# ollama run gemma

from pymilvus import MilvusClient, FieldSchema, DataType, Collection, connections
import ollama

conversation = [
    "Alice: Hey, how was your weekend?",
    "Bob: Pretty good! I went hiking. You?",
    "Alice: Nice! I just relaxed and watched a few movies.",
    "Bob: That sounds great. Anything worth recommending?",
    "Alice: Yeah, 'The Secret Garden' was surprisingly good!",
    "Bob: I'll check it out. Thanks!"
]

records_dict = []
for i, line in enumerate(conversation):
    records_dict.append(
        {
            "id": i+1,
            "original_text": line,
            "vector": ollama.embeddings(model='nomic-embed-text', prompt=line)['embedding']
        }
    )
    
# print(records_dict)
print("part 1 done")

part 1 done


In [13]:
# create a milvus collection
client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)
print(client)

collection_name = "test_textvector_storage"
exists = client.has_collection(collection_name=collection_name)
print(f"exists:{exists}")

def create_collection(client):
    # 3.1. Create schema
    schema = MilvusClient.create_schema(
        enable_dynamic_field=True,
    )
    
    # 3.2. Add fields to schema
    schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
    schema.add_field(field_name="original_text", datatype=DataType.VARCHAR, max_length=512)
    schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=768)
    
    # 3.3. Prepare index parameters
    index_params = client.prepare_index_params()
    
    # 3.4. Add indexes
    index_params.add_index(
        field_name="id",
        index_type="AUTOINDEX"
    )
    
    index_params.add_index(
        field_name="vector", 
        index_type="AUTOINDEX",
        metric_type="COSINE"
    )
    
    return client.create_collection(
        collection_name=collection_name,
        schema=schema,
        index_params=index_params
    )
        
if not exists:
    print(create_collection(client))

connections.connect(alias="default", host="127.0.0.1", port="19530")

collection = Collection(collection_name)
collection.load()
print(f"collection: {collection}")

exists:True
collection: <Collection>:
-------------
<name>: test_textvector_storage
<description>: 
<schema>: {'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'original_text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'enable_dynamic_field': True}



In [3]:
# now to insert
res = client.insert(
    collection_name=collection_name,
    data=records_dict
)

In [34]:
# now retrieve the documents based on prompt
# question = "what did everyone do in their weekend?"
question = "Recommend me something based on what they said"
prompt_vector = ollama.embeddings(model='nomic-embed-text', prompt=question)['embedding']

res = collection.search(
    anns_field="vector",
    data=[prompt_vector],
    param={"nprobe": 16},
    limit=3,
    output_fields=["original_text"]
)


# fusing the gathered information
retrieved_documents = ""
for hits in res:
    for hit in hits:
        retrieved_documents = retrieved_documents + hit['entity']['original_text'] + ";"
        print(hit)

# print(retrieved_documents)


{'id': 4, 'distance': 0.6763947606086731, 'entity': {'original_text': 'Bob: That sounds great. Anything worth recommending?'}}
{'id': 6, 'distance': 0.5386989712715149, 'entity': {'original_text': "Bob: I'll check it out. Thanks!"}}
{'id': 2, 'distance': 0.494762122631073, 'entity': {'original_text': 'Bob: Pretty good! I went hiking. You?'}}


In [35]:
response = ollama.chat(model='gemma:7b', messages=[{
    'role': 'user', 
    'content': f"Answer this question: {question} Based on the information here: {retrieved_documents}",
}])
print(response['message']['content'])

Based on Bob's responses, it's not possible to recommend anything as the conversation does not provide any context or preferences regarding recommendations.
